In [0]:
import keras

In [0]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Input, Concatenate, AveragePooling2D
from keras.optimizers import adam, sgd
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
import numpy as np

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
def inception_module(prev_layer):
    #main_input = Input(shape = input_shape, d_type = 'float32')
    x1 = Conv2D(filters = 64, kernel_size = (1,1), padding = 'same')(prev_layer)
    x2 = Conv2D(filters = 64, kernel_size = (3,3), padding = 'same')(prev_layer)
    x3 = Conv2D(filters = 64, kernel_size = (5,5), padding = 'same')(prev_layer)
    x4 = MaxPool2D(strides = (1,1),padding = 'same')(prev_layer)
    concat = keras.layers.concatenate([x1,x2,x3,x4] , axis = 3) 
    return concat

In [0]:
def create_googleNet(input_shape, num_classes):
    main_input = Input(shape = input_shape, dtype = 'float32')
    conv1 = Conv2D(15, kernel_size = (3,3))(main_input)
    prev_layer = conv1
    
    for i in range(3):
        x = inception_module(prev_layer)
        x2 = MaxPool2D()(x)
        prev_layer = x2
        
    avgpool = AveragePooling2D()(prev_layer)
    flat = Flatten()(avgpool)
    den1 = Dense(64, activation='relu')(flat)
    main_output = Dense(num_classes, activation = 'softmax')(den1)
    model = Model(inputs=[main_input], outputs=[main_output])
    return model

In [0]:
num_classes = np.max(y_train) - np.min(y_train) + 1
batch_size = 128
epochs = 20

In [0]:
x_train = x_train.astype('float32') 
x_test = x_test.astype('float32') 
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [0]:
model = create_googleNet(x_train[0].shape, num_classes)

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = adam(), metrics = ['accuracy'])

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 30, 30, 15)   420         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 30, 30, 64)   1024        conv2d_21[0][0]                  
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 30, 30, 64)   8704        conv2d_21[0][0]                  
__________________________________________________________________________________________________
conv2d_24 

In [29]:
history = model.fit(x_train, y_train,
                                batch_size = batch_size,
                                epochs = epochs,
                                verbose = 1,
                                validation_data = (x_test,y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 44s 888us/step - loss: 1.6858 - acc: 0.3868 - val_loss: 1.2944 - val_acc: 0.5430
Epoch 2/20
50000/50000 [==============================] - 44s 876us/step - loss: 1.1528 - acc: 0.5947 - val_loss: 1.0457 - val_acc: 0.6385
Epoch 3/20
50000/50000 [==============================] - 44s 876us/step - loss: 0.9619 - acc: 0.6691 - val_loss: 0.9568 - val_acc: 0.6757
Epoch 4/20
50000/50000 [==============================] - 44s 878us/step - loss: 0.8430 - acc: 0.7116 - val_loss: 0.9388 - val_acc: 0.6832
Epoch 5/20
50000/50000 [==============================] - 44s 874us/step - loss: 0.7407 - acc: 0.7451 - val_loss: 0.8819 - val_acc: 0.7047
Epoch 6/20
50000/50000 [==============================] - 44s 876us/step - loss: 0.6536 - acc: 0.7752 - val_loss: 0.8470 - val_acc: 0.7258
Epoch 7/20
50000/50000 [==============================] - 44s 878us/step - loss: 0.5627 - acc: 0.8059 - val_loss: 0.